In [2]:
def solve():
    import sys
    from twocaptcha import TwoCaptcha
    result = None

    sitekey = '<your_site_key_from_two_captcha>'
    api_key = '<your_api_key_from_two_captcha>'
    solver = TwoCaptcha(api_key)
    try:
        result = solver.solve_captcha(
            sitekey=sitekey,
            url='https://www.deezer.com/en/login'
        )

    except Exception as e:
        sys.exit(e)

    return result

In [7]:

textarea = driver.find_element('xpath','//*[@id="g-recaptcha-response"]')
solution = solve()
code = solution['code']
driver.execute_script(f"var ele=arguments[0]; ele.innerHTML = '{code}';", textarea)
time.sleep(1) # waiting is mandatory
textarea = driver.find_element_by_xpath('//*[@id="login-btn"]/button').click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="g-recaptcha-response"]"}
  (Session info: chrome=105.0.5195.102)
Stacktrace:
#0 0x562795d08693 <unknown>
#1 0x562795b01b0a <unknown>
#2 0x562795b3a5f7 <unknown>
#3 0x562795b3a7c1 <unknown>
#4 0x562795b6d804 <unknown>
#5 0x562795b5794d <unknown>
#6 0x562795b6b4b0 <unknown>
#7 0x562795b57743 <unknown>
#8 0x562795b2d533 <unknown>
#9 0x562795b2e715 <unknown>
#10 0x562795d587bd <unknown>
#11 0x562795d5bbf9 <unknown>
#12 0x562795d3df2e <unknown>
#13 0x562795d5c9b3 <unknown>
#14 0x562795d31e4f <unknown>
#15 0x562795d7bea8 <unknown>
#16 0x562795d7c052 <unknown>
#17 0x562795d9671f <unknown>
#18 0x7f9b83a2b609 <unknown>


In [1]:
from fake_useragent import UserAgent
ua = UserAgent()

In [11]:
userAgent = ua.random

In [12]:
userAgent

'Mozilla/4.79 [en] (compatible; MSIE 7.0; Windows NT 5.0; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 1.1.4322; .NET CLR 3.0.04506.30; .NET CLR 3.0.04506.648)'

In [5]:
import pandas as pd
import time
import re
from IPython.display import clear_output

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from agenda_tools.get_schedule import remove_blank_spaces
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities


from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementNotInteractableException
from concurrent.futures import ThreadPoolExecutor

In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
# options.add_argument('--headless')
options.add_argument("--incognito")

prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:107.0) Gecko/20100101 Firefox/107.0")


capa = DesiredCapabilities.CHROME
capa["pageLoadStrategy"] = "none"

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options, desired_capabilities=capa)
driver.get("https://www.google.com")
wait = WebDriverWait(driver, 3)

In [3]:
driver.get('https://www.tdlr.texas.gov/TABS/Search/Project/TABS2023006663')

## Process

In [4]:
def get_text(path):
    xpath = f'//dt[contains(.,"{path}")]/following-sibling::dd'
    return driver.find_element('xpath',xpath).text

In [19]:
driver.current_url

'https://www.tdlr.texas.gov/TABS/Search/Project/TABS2023006680'

In [29]:
def scrape(driver):
    time.sleep(0.5)
    try:
        wait.until(EC.presence_of_element_located((By.XPATH, f'//dt[contains(.,"Project Name")]/following-sibling::dd')))
    except TimeoutException:
        return None
        
    try:
        project_name = get_text('Project Name')
    except NoSuchElementException:
        return None
    url_link = driver.current_url,
    project_number = get_text('Project Number:')
    facility_name = get_text('Facility Name:')
    address = get_text('Location Address:')
    location = get_text('Location County:')
    start_date = get_text('Start Date:')
    completation_date = get_text('Completion Date:')
    estimated_cost = get_text('Estimated Cost:')
    type_work = get_text('Type of Work:')
    type_funds = get_text('Type of Funds:')
    scope_work = get_text('Scope of Work:')
    square_footage = get_text('Square Footage:')
    are_private = get_text('Are the private funds provided by the tenant?')
    current_status = get_text('Current Status:')
    
    return {
        'URL':url_link,
        'Project Name':project_name,
        'Project Number:':project_number,
        'Facility Name:':facility_name,
        'Location Address:':address,
        'Location County:':location,
        'Start Date:':start_date,
        'Completion Date:':completation_date,
        'Estimated Cost:':estimated_cost,
        'Type of Work:':type_work,
        'Type of Funds:':type_funds,
        'Scope of Work:':scope_work,
        'Square Footage:':square_footage,
        'Are the private funds provided by the tenant?':are_private,
        'Current Status:':current_status
        
    }


In [6]:
list_url = []

In [7]:
for i in range(60,81):
    url = f'https://www.tdlr.texas.gov/TABS/Search/Project/TABS20230066{i}'
    list_url.append(url)

In [30]:
list_data = []
for i in list_url:
    driver.get(i)
    data = scrape(driver)
    if data:
        list_data.append(data)

In [31]:
df = pd.DataFrame(list_data)

In [32]:
df.head()

,URL,Project Name,Project Number:,Facility Name:,Location Address:,Location County:,Start Date:,Completion Date:,Estimated Cost:,Type of Work:,Type of Funds:,Scope of Work:,Square Footage:,Are the private funds provided by the tenant?,Current Status:
0,(https://www.tdlr.texas.gov/TABS/Search/Projec...,Bombay Vapor,TABS2023006660,Northlake Plaza Property LLC,"5000 TX-114, Unit 110",Denton,1/2/2023,3/3/2023,"$120,000",Renovation/Alteration,"This project is privately funded, on private l...",This is an interior finish-out in an existing ...,"2,257 ft 2",No,Project Registered
1,(https://www.tdlr.texas.gov/TABS/Search/Projec...,Cidercade Arlington,TABS2023006661,Cidercade Arlington,500 E Division St,Tarrant,12/15/2021,4/1/2022,"$600,000",Renovation/Alteration,"This project is privately funded, on private l...",Interior remodel for indoor commercial amuseme...,"25,482 ft 2",Yes,Project Registered
2,(https://www.tdlr.texas.gov/TABS/Search/Projec...,Post Oak Centre North Suite 302,TABS2023006662,Post Oak Centre North,"2200 Park Bend Dr. Building 1, Suite 302",Travis,2/20/2023,8/21/2023,"$161,700",Renovation/Alteration,"This project is privately funded, on private l...",Renovation of existing office suite for use as...,"1,617 ft 2",Yes,Project Registered
3,(https://www.tdlr.texas.gov/TABS/Search/Projec...,Floss Dental,TABS2023006663,Floss Dental,2575 Town Center N Blvd. Suite L,Fort Bend,1/2/2023,4/3/2023,"$300,000",Renovation/Alteration,"This project is privately funded, on private l...",Dental Office buildout,"3,046 ft 2",Yes,Project Registered
4,(https://www.tdlr.texas.gov/TABS/Search/Projec...,Elevator Installation and Refurbishment,TABS2023006664,Rick Husband Amarillo International Airport-AM...,10801 Airport Boulevard,Potter,12/2/2022,1/2/2024,"$1,250,000",Renovation/Alteration,"This project involves public funds, public lan...",THE PROPOSED BASE BID WORK INCLUDES A NEW TRAC...,666 ft 2,No,Project Registered


In [18]:
df.to_excel('data_test.xlsx', encoding='UTF-8', index=False)